In [1]:
from dotenv import load_dotenv


load_dotenv()

True

In [2]:
import os, sys
from pathlib import Path


def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")


src_path = find_src_folder()
sys.path.append(str(src_path))

## Super Dungeon Agent 테스트 (Monster Agent 통합)

In [3]:
from agents.dungeon.super.dungeon_agent import create_super_dungeon_graph
import json

# Super Dungeon Graph 생성
super_graph = create_super_dungeon_graph()
print("✅ Super Dungeon Graph 생성 완료")

✅ Super Dungeon Graph 생성 완료


### 테스트 실행

In [4]:
from agents.dungeon.monster.monster_database import MONSTER_DATABASE

# 테스트 State 구성
test_state = {
    "dungeon_base_data": {
        "dungeon_id": 1,
        "floor_count": 3,
        "rooms": [
            {"room_id": 1, "room_type": "start", "position": {"x": 0.0, "y": 0.0}},
            {"room_id": 2, "room_type": "monster", "position": {"x": 0.2, "y": 0.1}},
            {"room_id": 3, "room_type": "monster", "position": {"x": 0.4, "y": 0.2}},
            {"room_id": 4, "room_type": "treasure", "position": {"x": 0.6, "y": 0.3}},
            {"room_id": 5, "room_type": "monster", "position": {"x": 0.8, "y": 0.4}},
            {"room_id": 6, "room_type": "boss", "position": {"x": 0.9, "y": 0.5}},
        ],
    },
    "heroine_data": {
        "heroine_id": 1,
        "name": "레티아",
        "event_room": 3,
        "memory_progress": 50,
    },
    "heroine_stat": {
        "hp": 500,
        "strength": 20,
        "dexterity": 15,
        "intelligence": 10,
        "defense": 10,
        "critChance": 15.0,
        "attackSpeed": 1.5,
        "moveSpeed": 400,
        "skillDamageMultiplier": 1.2,
    },
    "heroine_memories": [],
    "monster_db": MONSTER_DATABASE,
    "dungeon_player_data": {
        "affection": 50,
        "sanity": 80,
        "difficulty_level": "normal",
    },
    "used_events": [],
    "event_result": {},
    "filled_dungeon_data": {},
    "difficulty_log": {},
    "final_dungeon_json": {},
}

print("=" * 70)
print("Super Dungeon Agent 테스트 시작")
print("=" * 70)

# Graph 실행
result = super_graph.invoke(test_state)

print("\n" + "=" * 70)
print("✅ 실행 완료")
print("=" * 70)

Super Dungeon Agent 테스트 시작

[Event Node] 이벤트 생성 시작...

[Monster Node] 몬스터 밸런싱 시작...
[calculate_combat_score_node] 플레이어 전투력: 194.00
[calculate_combat_score_node] HP: 500, STR: 20, DEX: 15
[llm_strategy_node DEBUG] combat_score: 194.0
[llm_strategy_node DEBUG] heroine_stat type: <class 'dict'>
[llm_strategy_node DEBUG] heroine_stat: {'hp': 500, 'strength': 20, 'dexterity': 15, 'intelligence': 10, 'defense': 10, 'critChance': 15.0, 'attackSpeed': 1.5, 'moveSpeed': 400, 'skillDamageMultiplier': 1.2}
[llm_strategy_node DEBUG] dungeon_player_data: {'affection': 50, 'sanity': 80, 'difficulty_level': 'normal'}
[llm_strategy_node DEBUG] hero_summary:

전투 스탯:
- HP: 500
- 근력(Strength): 20
- 기량(Dexterity): 15
- 치명타 확률: 15.0%
- 공격 속도: 1.50x
- 스킬 데미지 배율: 1.20x
- 종합 전투력: 194.0 (플레이어)

던전 진행 상황:
- 현재 층: 3
- 난이도: 1
- 호감도: 50
- 정신력: 80

[llm_strategy_node DEBUG] prompts type: <class 'str'>
[llm_strategy_node DEBUG] prompts 내용:
<세계관>
레테(Lethe)는 기억과 망각이 섞인 중세 판타지 행성입니다. ‘암네시아’ 재앙 이후 기억을 잃은 망각자들은 특별한 능력을 얻

### 결과 확인

In [5]:
final_json = result.get("final_dungeon_json", {})

print("\n📊 최종 결과:")
print(json.dumps(final_json, indent=2, ensure_ascii=False))
print("\n\n🎯 요약:")
print(f"Event - Main: {final_json.get('events', {}).get('main_event', 'N/A')[:50]}...")
print(f"Monster - 총 {final_json.get('monster_stats', {}).get('total_count', 0)}마리")
print(f"  - 보스: {final_json.get('monster_stats', {}).get('boss_count', 0)}마리")
print(f"  - 일반: {final_json.get('monster_stats', {}).get('normal_count', 0)}마리")
print(
    f"Difficulty - x{final_json.get('difficulty_info', {}).get('ai_multiplier', 0):.2f}"
)
print(
    f"Combat Score - {final_json.get('difficulty_info', {}).get('combat_score', 0):.2f}"
)

print("\n\n🏰 방별 몬스터 배치 (언리얼 엔진용):")
for room in final_json.get("dungeon_data", {}).get("rooms", []):
    room_type = room.get("room_type")
    monsters = room.get("monsters", [])
    if monsters:
        print(f"\n방 #{room['room_id']} ({room_type}):")
        for m in monsters:
            position = m.get("position", {})
            print(
                f"  - 몬스터 ID: {m['monster_id']}, 이름: {m['monster_name']}, 타입: {m['monster_type']}, 위협도: {m['threat_level']:.2f}"
            )
            print(
                f"    좌표: x={position.get('x', 0):.2f}, y={position.get('y', 0):.2f}, z={position.get('z', 0):.2f}"
            )


📊 최종 결과:
{
  "dungeon_data": {
    "dungeon_id": 1,
    "floor_count": 3,
    "rooms": [
      {
        "room_id": 1,
        "room_type": "start"
      },
      {
        "room_id": 2,
        "room_type": "monster",
        "monsters": [
          {
            "monster_id": 0,
            "monster_name": "스켈레톤",
            "monster_type": 0,
            "position": {
              "x": 0.3626923749150742,
              "y": 0.1867274211539568,
              "z": 0.169940918226681
            },
            "hp": 300,
            "attack": 10,
            "threat_level": 105.0
          }
        ]
      },
      {
        "room_id": 3,
        "room_type": "monster",
        "monsters": [
          {
            "monster_id": 0,
            "monster_name": "스켈레톤",
            "monster_type": 0,
            "position": {
              "x": 0.7480029340644256,
              "y": 0.027919879946912807,
              "z": 0.22492476790845561
            },
            "hp": 300,
     

## 멀티플레이어 테스트 (같은 던전, 다른 히로인)

멀티플레이어 환경에서 **같은 던전을 함께 이동**하며 **같은 이벤트**를 경험합니다.
- **같은 던전 구조**: 모든 플레이어가 동일한 방 배치와 몬스터를 만남
- **같은 이벤트**: 모든 플레이어가 같은 메인 이벤트 발생
- **개별 텍스트**: 개별 이벤트일 경우, 각 히로인에 맞는 고유한 서브 이벤트 텍스트 표시

In [6]:
# 멀티플레이어용 던전 베이스 데이터 (모든 플레이어가 공유)
shared_dungeon_base = {
    "dungeon_id": 1,
    "floor_count": 3,
    "rooms": [
        {"room_id": 1, "room_type": "start"},
        {"room_id": 2, "room_type": "monster"},
        {"room_id": 3, "room_type": "monster"},
        {"room_id": 4, "room_type": "treasure"},
        {"room_id": 5, "room_type": "monster"},
        {"room_id": 6, "room_type": "boss"},
    ],
}

# 플레이어별 히로인 설정 (같은 던전, 다른 히로인)
# 모든 플레이어가 같은 event_room에서 이벤트를 경험
shared_event_room = 3  # 모든 플레이어가 3번 방에서 같은 이벤트 발생

players = [
    {
        "player_id": 101,
        "heroine_data": {
            "heroine_id": 1,  # 레티아
            "name": "레티아",
            "event_room": shared_event_room,  # 같은 방에서 이벤트 발생
            "memory_progress": 50,
        },
        "heroine_stat": {
            "hp": 500,
            "strength": 20,
            "dexterity": 15,
            "intelligence": 10,
            "defense": 10,
            "critChance": 15.0,
            "attackSpeed": 1.5,
            "moveSpeed": 400,
            "skillDamageMultiplier": 1.2,
        },
        "dungeon_player_data": {
            "affection": 50,
            "sanity": 80,
            "difficulty_level": "normal",
        },
    },
    {
        "player_id": 102,
        "heroine_data": {
            "heroine_id": 2,
            "name": "루파메스",
            "event_room": shared_event_room,  # 같은 방에서 이벤트 발생
            "memory_progress": 30,
        },
        "heroine_stat": {
            "hp": 450,
            "strength": 25,
            "dexterity": 18,
            "intelligence": 8,
            "defense": 12,
            "critChance": 20.0,
            "attackSpeed": 1.8,
            "moveSpeed": 420,
            "skillDamageMultiplier": 1.0,
        },
        "dungeon_player_data": {
            "affection": 70,
            "sanity": 60,
            "difficulty_level": "normal",  # 같은 난이도
        },
    },
]

print("✅ 멀티플레이어 테스트 데이터 준비 완료")
print(f"총 {len(players)}명의 플레이어")
print(f"공유 던전 ID: {shared_dungeon_base['dungeon_id']}")
print(f"공유 이벤트 발생 방: Room #{shared_event_room}")

✅ 멀티플레이어 테스트 데이터 준비 완료
총 2명의 플레이어
공유 던전 ID: 1
공유 이벤트 발생 방: Room #3


### 공유 던전 생성 (1회만 실행)

먼저 **하나의 던전**을 생성합니다. 이 던전은 모든 플레이어가 공유합니다.
- 몬스터 배치는 첫 번째 플레이어 기준으로 생성
- 이벤트는 첫 번째 플레이어 기준으로 선택

In [7]:
# Step 1: 공유 던전 생성 (첫 번째 플레이어 기준)
print("\n" + "=" * 80)
print("🏰 공유 던전 생성 시작 (모든 플레이어가 사용할 던전)")
print("=" * 80)

# 첫 번째 플레이어 정보로 던전 생성
first_player = players[0]
print(
    f"기준 플레이어: {first_player['player_id']} - {first_player['heroine_data']['name']}"
)

shared_dungeon_state = {
    "dungeon_base_data": shared_dungeon_base,
    "heroine_data": first_player["heroine_data"],
    "heroine_stat": first_player["heroine_stat"],
    "heroine_memories": [],
    "monster_db": MONSTER_DATABASE,
    "dungeon_player_data": first_player["dungeon_player_data"],
    "used_events": [],
    "event_result": {},
    "filled_dungeon_data": {},
    "difficulty_log": {},
    "final_dungeon_json": {},
}

# 공유 던전 생성
shared_dungeon_result = super_graph.invoke(shared_dungeon_state)
shared_final_json = shared_dungeon_result.get("final_dungeon_json", {})

print("\n✅ 공유 던전 생성 완료")
print(f"   던전 ID: {shared_dungeon_base['dungeon_id']}")
print(
    f"   총 몬스터: {shared_final_json.get('monster_stats', {}).get('total_count', 0)}마리"
)
print(
    f"   메인 이벤트: {shared_final_json.get('events', {}).get('main_event', 'N/A')[:50]}..."
)
print("=" * 80)


🏰 공유 던전 생성 시작 (모든 플레이어가 사용할 던전)
기준 플레이어: 101 - 레티아

[Event Node] 이벤트 생성 시작...
[heroine_memories_node] 히로인 ID: 1
[heroine_memories_node] 기억 진척도: 50
[heroine_memories_node] 해금된 기억 개수: 2
[selected_main_event_node] 선택된 이벤트: 미지의 기억
[selected_main_event_node] 개별 이벤트 여부: False

[Monster Node] 몬스터 밸런싱 시작...
[calculate_combat_score_node] 플레이어 전투력: 194.00
[calculate_combat_score_node] HP: 500, STR: 20, DEX: 15
[llm_strategy_node DEBUG] combat_score: 194.0
[llm_strategy_node DEBUG] heroine_stat type: <class 'dict'>
[llm_strategy_node DEBUG] heroine_stat: {'hp': 500, 'strength': 20, 'dexterity': 15, 'intelligence': 10, 'defense': 10, 'critChance': 15.0, 'attackSpeed': 1.5, 'moveSpeed': 400, 'skillDamageMultiplier': 1.2}
[llm_strategy_node DEBUG] dungeon_player_data: {'affection': 50, 'sanity': 80, 'difficulty_level': 'normal'}
[llm_strategy_node DEBUG] hero_summary:

전투 스탯:
- HP: 500
- 근력(Strength): 20
- 기량(Dexterity): 15
- 치명타 확률: 15.0%
- 공격 속도: 1.50x
- 스킬 데미지 배율: 1.20x
- 종합 전투력: 194.0 (플레이어)

던

### 각 플레이어별 히로인 반응 텍스트 생성

**이벤트 구조:**
1. 메인 이벤트: 모든 플레이어 공통 (같은 텍스트)
2. 서브 이벤트: 모든 플레이어 공통 (같은 진행 과정)
3. 행동 선택지: 모든 플레이어 공통
4. **히로인 반응**: 각 히로인의 시나리오/해금 정보에 따라 개별 생성 ← 이 부분만 개별화!

In [15]:
# Step 2: 공유 이벤트 데이터 + 각 히로인별 반응 텍스트 생성
print("\n" + "=" * 80)
print("📝 공유 이벤트 + 히로인별 반응 텍스트 생성")
print("=" * 80)

# 공유 던전 데이터와 이벤트 정보 (모든 플레이어 공통)
shared_dungeon_data = shared_final_json.get("dungeon_data", {})
shared_events = shared_final_json.get("events", {})

# 공통 이벤트 정보
common_event_data = {
    "main_event": shared_events.get("main_event", ""),  # 공통
    "sub_event": shared_events.get("sub_event", ""),  # 공통
    "event_room_index": shared_events.get("event_room_index", -1),  # 공통
    "final_answer": shared_events.get("final_answer", ""),  # 이벤트 선택 이유
}

print(f"\n📌 공통 메인 이벤트: {common_event_data['main_event'][:80]}...")
print(f"📌 공통 서브 이벤트: {common_event_data['sub_event'][:80]}...")
print(f"📌 이벤트 발생 방: Room #{common_event_data['event_room_index']}")

multiplayer_results = {}

# 히로인 반응 생성기 (히로인 ID/기억 진척도 기반으로 차별화)
def generate_heroine_reaction(heroine_id: int, memory_progress: int, heroine_name: str) -> str:
    # 간단한 차별화 규칙: 히로인별 톤/키워드 + 기억 단계별 감정 변화
    tone_map = {
        1: "따뜻하고 결연한",   # 레티아
        2: "차분하고 분석적인", # 루파메스
        3: "쾌활하고 직감적인",
    }
    keyword_map = {
        1: ["약속", "보호", "용기"],
        2: ["기록", "가설", "증거"],
        3: ["우정", "모험", "직감"],
    }
    stage = "초기" if memory_progress < 25 else ("중기" if memory_progress < 60 else "후기")
    tone = tone_map.get(heroine_id, "담담한")
    keywords = keyword_map.get(heroine_id, ["기억", "감정", "결심"])
    focus = keywords[memory_progress % len(keywords)]

    return (
        f"[{heroine_name}의 반응] {tone} 어조로 {stage} 기억을 되짚는다. "
        f"이번 사건은 그녀의 '{focus}'과(와) 밀접하게 연결되어 있었고, "
        f"이로 인해 다음 선택지에서 더 적극적으로 개입하려는 결심을 굳혔다."
    )

# 각 히로인별 반응 텍스트 개별 생성
for player in players:
    player_id = player["player_id"]
    heroine_name = player["heroine_data"]["name"]
    heroine_id = player["heroine_data"]["heroine_id"]
    memory_progress = player["heroine_data"]["memory_progress"]

    print(f"\n{'─' * 80}")
    print(
        f"🎮 Player {player_id} - {heroine_name} (ID: {heroine_id}, 기억진척도: {memory_progress})"
    )

    # 히로인별 개별 반응 텍스트 생성 (차별화 적용)
    heroine_reaction = generate_heroine_reaction(heroine_id, memory_progress, heroine_name)

    # 플레이어별 결과 저장
    player_result = {
        "final_dungeon_json": {
            "dungeon_data": shared_dungeon_data,  # 공통 던전
            "events": {
                **common_event_data,  # 공통 이벤트
                "heroine_reaction": heroine_reaction,  # 히로인별 개별 반응
            },
            "monster_stats": shared_final_json.get("monster_stats", {}),  # 공통 몬스터
            "difficulty_info": shared_final_json.get("difficulty_info", {}),
        }
    }

    multiplayer_results[player_id] = player_result
    print(f"   ✓ 히로인 반응 텍스트: {heroine_reaction}")

print("\n" + "=" * 80)
print(f"✅ 전체 {len(multiplayer_results)}명의 플레이어 반응 텍스트 생성 완료")
print("=" * 80)


📝 공유 이벤트 + 히로인별 반응 텍스트 생성

📌 공통 메인 이벤트: [미지의 기억]
Event Code: UNKNOWN_MEMORY
Type: 공통 이벤트

방에 들어가는 순간, 당신은 갑자기 머리속이 흐릿해지며...
📌 공통 서브 이벤트: 
=== 서브 이벤트 내러티브 ===
발을 들여놓는 순간, 방의 윤곽이 물처럼 흐트러지며 시야가 뒤집힌다. 차가운 돌바닥의 감촉이 사라지고, 대...
📌 이벤트 발생 방: Room #3

────────────────────────────────────────────────────────────────────────────────
🎮 Player 101 - 레티아 (ID: 1, 기억진척도: 50)
   ✓ 히로인 반응 텍스트: [레티아의 반응] 따뜻하고 결연한 어조로 중기 기억을 되짚는다. 이번 사건은 그녀의 '용기'과(와) 밀접하게 연결되어 있었고, 이로 인해 다음 선택지에서 더 적극적으로 개입하려는 결심을 굳혔다.

────────────────────────────────────────────────────────────────────────────────
🎮 Player 102 - 루파메스 (ID: 2, 기억진척도: 30)
   ✓ 히로인 반응 텍스트: [루파메스의 반응] 차분하고 분석적인 어조로 중기 기억을 되짚는다. 이번 사건은 그녀의 '기록'과(와) 밀접하게 연결되어 있었고, 이로 인해 다음 선택지에서 더 적극적으로 개입하려는 결심을 굳혔다.

✅ 전체 2명의 플레이어 반응 텍스트 생성 완료


### 멀티플레이어 이벤트 검증

**검증 항목:**
1. ✅ 모든 플레이어가 같은 메인 이벤트를 경험
2. ✅ 모든 플레이어가 같은 서브 이벤트를 경험
3. ✅ 각 히로인별로 다른 반응 텍스트를 받음

In [9]:
print("\n" + "🎭 " + "=" * 76 + " 🎭")
print("                    멀티플레이어 이벤트 검증")
print("🎭 " + "=" * 76 + " 🎭\n")

# 1. 공통 이벤트 확인
print("=" * 80)
print("1️⃣  공통 이벤트 데이터 (모든 플레이어가 동일하게 경험)")
print("=" * 80)

first_result = list(multiplayer_results.values())[0]
common_main = first_result["final_dungeon_json"]["events"]["main_event"]
common_sub = first_result["final_dungeon_json"]["events"]["sub_event"]
common_room = first_result["final_dungeon_json"]["events"]["event_room_index"]

print(f"\n📌 메인 이벤트 (공통):")
print(f"   {common_main}\n")

print(f"💬 서브 이벤트 (공통):")
print(f"   {common_sub}\n")

print(f"🏠 이벤트 발생 방: Room #{common_room}")

# 2. 각 플레이어별 히로인 반응 확인
print("\n\n" + "=" * 80)
print("2️⃣  히로인별 개별 반응 텍스트")
print("=" * 80)

for player_id, result in multiplayer_results.items():
    events = result["final_dungeon_json"]["events"]
    player_info = next(p for p in players if p["player_id"] == player_id)
    heroine_name = player_info["heroine_data"]["name"]
    heroine_id = player_info["heroine_data"]["heroine_id"]

    print(f"\n{'─' * 80}")
    print(f"🎮 Player {player_id} - {heroine_name} (ID: {heroine_id})")
    print(f"{'─' * 80}")

    heroine_reaction = events.get("heroine_reaction", "N/A")
    print(f"💖 히로인 반응:")
    print(f"   {heroine_reaction}")

# 3. 검증
print("\n\n" + "🔍 " + "=" * 76 + " 🔍")
print("                      이벤트 구조 검증")
print("🔍 " + "=" * 76 + " 🔍\n")

# 메인/서브 이벤트가 모두 동일한지 확인
main_events = [
    r["final_dungeon_json"]["events"]["main_event"]
    for r in multiplayer_results.values()
]
sub_events = [
    r["final_dungeon_json"]["events"]["sub_event"] for r in multiplayer_results.values()
]
heroine_reactions = [
    r["final_dungeon_json"]["events"]["heroine_reaction"]
    for r in multiplayer_results.values()
]

print("✅ 메인 이벤트 검증:")
if len(set(main_events)) == 1:
    print("   → ✅ 모든 플레이어가 같은 메인 이벤트를 경험합니다!")
else:
    print("   → ❌ 메인 이벤트가 플레이어마다 다릅니다!")

print("\n✅ 서브 이벤트 검증:")
if len(set(sub_events)) == 1:
    print("   → ✅ 모든 플레이어가 같은 서브 이벤트를 경험합니다!")
else:
    print("   → ❌ 서브 이벤트가 플레이어마다 다릅니다!")

print("\n✅ 히로인 반응 검증:")
if len(set(heroine_reactions)) == len(heroine_reactions):
    print(
        f"   → ✅ 각 히로인별로 고유한 반응 텍스트! ({len(set(heroine_reactions))}/{len(heroine_reactions)})"
    )
elif len(set(heroine_reactions)) == 1:
    print("   → ⚠️  모든 히로인의 반응이 동일합니다!")
else:
    print(
        f"   → ℹ️  일부 고유한 반응 텍스트 ({len(set(heroine_reactions))}/{len(heroine_reactions)})"
    )

print("\n" + "🎭 " + "=" * 76 + " 🎭")


🎭 ============================================================================ 🎭
                    멀티플레이어 이벤트 검증
🎭 ============================================================================ 🎭

1️⃣  공통 이벤트 데이터 (모든 플레이어가 동일하게 경험)

📌 메인 이벤트 (공통):
   [미지의 기억]
Event Code: UNKNOWN_MEMORY
Type: 공통 이벤트

방에 들어가는 순간, 당신은 갑자기 머리속이 흐릿해지며 이전에 경험해보지 못한 상황이 눈앞에 펼쳐진다. 머리속에서 펼쳐지는 미지의 기억 속에서, 당신은 어떻게 행동할 것인지 결정해야 한다. 중요한 것은 그 기억 속에서 탈출해 정신을 차리는 것이다.

💬 서브 이벤트 (공통):
   
=== 서브 이벤트 내러티브 ===
발을 들여놓는 순간, 방의 윤곽이 물처럼 흐트러지며 시야가 뒤집힌다. 차가운 돌바닥의 감촉이 사라지고, 대신 안개 같은 어둠이 발목을 감싼다. 당신의 몸은 그대로인데, 의식만이 낯선 풍경 속으로 끌려 들어간다. 형체를 알 수 없는 수많은 장면이 번개처럼 스쳐 지나가며, 당신의 기억인지, 타인의 기억인지조차 구분되지 않는다. 이름 모를 목소리들이 겹겹이 속삭이고, 어떤 것은 달콤하게, 어떤 것은 위협적으로 당신을 붙잡으려 한다. 머릿속은 점점 더 흐려지고, 이 미지의 기억에서 벗어나지 못하면 자신이 누구였는지조차 잃어버릴 것 같은 공포가 엄습한다. 지금, 이 혼란스러운 기억 속에서 어떻게 움직일지 선택해야 한다.

=== 선택지 ===
1. action='머릿속에서 밀려드는 장면들을 억지로 끊고, 스스로의 이름과 현재 상황을 되뇌며 기억의 소용돌이를 강제로 밀어낸다.' reward_id='hp_recovery_all' penalty_id=None
2. action='눈을 감고 심호흡하며, 떠오르는 장면들을 하나씩 관찰

### 공유 던전 데이터 확인

모든 플레이어가 같은 던전 데이터를 사용하는지 확인합니다.
- 같은 방 구조
- 같은 몬스터 배치 (위치, 종류, 개수)

In [10]:
print("\n" + "🏰 " + "=" * 76 + " 🏰")
print("                      공유 던전 데이터 확인")
print("🏰 " + "=" * 76 + " 🏰\n")

# 모든 플레이어의 던전 데이터가 동일한지 확인
print("✅ 던전 데이터 동일성 검증:")

# 첫 번째 플레이어의 던전 데이터를 기준으로 비교
first_player_id = list(multiplayer_results.keys())[0]
reference_dungeon = multiplayer_results[first_player_id]["final_dungeon_json"][
    "dungeon_data"
]
reference_rooms = reference_dungeon.get("rooms", [])

print(f"\n기준 던전 (Player {first_player_id}):")
print(f"  - 총 방 개수: {len(reference_rooms)}개")
print(f"  - 던전 ID: {reference_dungeon.get('dungeon_id', 'N/A')}")

# 각 플레이어의 던전 데이터 비교
all_same = True
for player_id, result in multiplayer_results.items():
    dungeon_data = result["final_dungeon_json"]["dungeon_data"]
    rooms = dungeon_data.get("rooms", [])

    if len(rooms) != len(reference_rooms):
        print(f"❌ Player {player_id}: 방 개수 불일치!")
        all_same = False
    else:
        print(f"✓ Player {player_id}: 방 개수 일치 ({len(rooms)}개)")

if all_same:
    print("\n✅ 모든 플레이어가 같은 던전 구조를 공유합니다!")
else:
    print("\n⚠️  플레이어 간 던전 구조가 다릅니다!")

# 몬스터 배치 확인
print("\n\n🐉 공유 던전의 몬스터 배치:")
for room in reference_rooms:
    monsters = room.get("monsters", [])
    if monsters:
        print(f"\n  Room #{room['room_id']} ({room.get('room_type')}):")
        print(f"    몬스터 수: {len(monsters)}마리")
        for m in monsters:
            print(
                f"      - {m['monster_name']} (ID: {m['monster_id']}, 위협도: {m['threat_level']:.2f})"
            )

print("\n" + "🏰 " + "=" * 76 + " 🏰")


🏰 ============================================================================ 🏰
                      공유 던전 데이터 확인
🏰 ============================================================================ 🏰

✅ 던전 데이터 동일성 검증:

기준 던전 (Player 101):
  - 총 방 개수: 6개
  - 던전 ID: 1
✓ Player 101: 방 개수 일치 (6개)
✓ Player 102: 방 개수 일치 (6개)

✅ 모든 플레이어가 같은 던전 구조를 공유합니다!


🐉 공유 던전의 몬스터 배치:

  Room #2 (monster):
    몬스터 수: 2마리
      - 스켈레톤 (ID: 0, 위협도: 105.00)
      - 스켈레톤 (ID: 0, 위협도: 105.00)

  Room #6 (boss):
    몬스터 수: 1마리
      - 던전 보스 (ID: 100, 위협도: 2250.00)

🏰 ============================================================================ 🏰


### 이벤트 진행 흐름 시뮬레이션

실제 게임 플레이 시나리오를 시뮬레이션합니다:
1. 이벤트 발견 → 메인 이벤트 텍스트 표시 (공통)
2. 서브 이벤트 진행 → 행동 선택지 제시 (공통)
3. 플레이어 상호작용 → 보상 지급
4. **이벤트 클리어 → 각 히로인별 반응 표시 (개별)**

In [13]:
print("\n" + "🎬 " + "=" * 76 + " 🎬")
print("                   이벤트 진행 흐름 시뮬레이션")
print("🎬 " + "=" * 76 + " 🎬\n")

print("【 Room #3 진입 】")
print("모든 플레이어가 함께 Room #3에 도착했습니다.\n")

# Phase 1: 이벤트 발견 (공통)
print("=" * 80)
print("Phase 1: 이벤트 발견")
print("=" * 80)
print("\n💡 [게임 화면에 표시되는 공통 텍스트]")
print(f"\n{common_event_data['main_event']}\n")

# Phase 2: 서브 이벤트 진행 (공통)
print("\n" + "=" * 80)
print("Phase 2: 서브 이벤트 진행")
print("=" * 80)
print("\n📋 [게임 화면에 표시되는 공통 텍스트]")
print(f"\n{common_event_data['sub_event']}\n")

# Phase 3: 행동 선택지 (공통)
print("\n" + "=" * 80)
print("Phase 3: 행동 선택지 제시")
print("=" * 80)
print("\n🎯 [모든 플레이어에게 동일한 선택지]")
print("   1. 조사한다")
print("   2. 무시하고 지나간다")
print("   3. 신중하게 접근한다")
print("\n각 플레이어가 선택 및 상호작용...")
print("(플레이어 액션 입력 대기)\n")

# Phase 4: 이벤트 클리어 후 히로인별 반응 (개별)
print("\n" + "=" * 80)
print("Phase 4: 이벤트 클리어 - 히로인별 반응")
print("=" * 80)

for player_id, result in multiplayer_results.items():
    events = result["final_dungeon_json"]["events"]
    player_info = next(p for p in players if p["player_id"] == player_id)
    heroine_name = player_info["heroine_data"]["name"]

    print(f"\n{'─' * 80}")
    print(f"📱 Player {player_id}의 화면")
    print(f"{'─' * 80}")
    print(f"\n💖 [{heroine_name}의 반응]")
    print(f"{events.get('heroine_reaction', '')}")

print("\n\n" + "🎬 " + "=" * 76 + " 🎬")
print("                    시뮬레이션 완료")
print("🎬 " + "=" * 76 + " 🎬\n")

print("\n📝 요약:")
print("   ✓ 메인 이벤트: 모든 플레이어 공통")
print("   ✓ 서브 이벤트: 모든 플레이어 공통")
print("   ✓ 행동 선택지: 모든 플레이어 공통")
print("   ✓ 히로인 반응: 각 플레이어별 개별 표시")
print("   → 올바른 멀티플레이어 구조!")


🎬 ============================================================================ 🎬
                   이벤트 진행 흐름 시뮬레이션
🎬 ============================================================================ 🎬

【 Room #3 진입 】
모든 플레이어가 함께 Room #3에 도착했습니다.

Phase 1: 이벤트 발견

💡 [게임 화면에 표시되는 공통 텍스트]

[미지의 기억]
Event Code: UNKNOWN_MEMORY
Type: 공통 이벤트

방에 들어가는 순간, 당신은 갑자기 머리속이 흐릿해지며 이전에 경험해보지 못한 상황이 눈앞에 펼쳐진다. 머리속에서 펼쳐지는 미지의 기억 속에서, 당신은 어떻게 행동할 것인지 결정해야 한다. 중요한 것은 그 기억 속에서 탈출해 정신을 차리는 것이다.


Phase 2: 서브 이벤트 진행

📋 [게임 화면에 표시되는 공통 텍스트]


=== 서브 이벤트 내러티브 ===
발을 들여놓는 순간, 방의 윤곽이 물처럼 흐트러지며 시야가 뒤집힌다. 차가운 돌바닥의 감촉이 사라지고, 대신 안개 같은 어둠이 발목을 감싼다. 당신의 몸은 그대로인데, 의식만이 낯선 풍경 속으로 끌려 들어간다. 형체를 알 수 없는 수많은 장면이 번개처럼 스쳐 지나가며, 당신의 기억인지, 타인의 기억인지조차 구분되지 않는다. 이름 모를 목소리들이 겹겹이 속삭이고, 어떤 것은 달콤하게, 어떤 것은 위협적으로 당신을 붙잡으려 한다. 머릿속은 점점 더 흐려지고, 이 미지의 기억에서 벗어나지 못하면 자신이 누구였는지조차 잃어버릴 것 같은 공포가 엄습한다. 지금, 이 혼란스러운 기억 속에서 어떻게 움직일지 선택해야 한다.

=== 선택지 ===
1. action='머릿속에서 밀려드는 장면들을 억지로 끊고, 스스로의 이름과 현재 상황을 되뇌며 기억의 소용돌이를 강제로 밀어낸다.' reward_id='hp_recovery_a

In [14]:
print("\n" + "📊 " + "=" * 76 + " 📊")
print("                         최종 요약")
print("📊 " + "=" * 76 + " 📊\n")

print("┌" + "─" * 78 + "┐")
print("│" + " " * 20 + "멀티플레이어 이벤트 데이터 구조" + " " * 28 + "│")
print("├" + "─" * 78 + "┤")

# 공통 데이터
print("│ 【공통 데이터】" + " " * 63 + "│")
print(
    "│   • 던전 구조: "
    + f"{len(shared_dungeon_data.get('rooms', []))}개 방"
    + " " * 53
    + "│"
)
print(
    "│   • 몬스터 배치: "
    + f"{shared_final_json.get('monster_stats', {}).get('total_count', 0)}마리"
    + " " * 54
    + "│"
)
print(
    "│   • 메인 이벤트: "
    + common_event_data["main_event"][:40]
    + "..."
    + " " * 18
    + "│"
)
print(
    "│   • 서브 이벤트: " + common_event_data["sub_event"][:40] + "..." + " " * 18 + "│"
)
print("│" + " " * 78 + "│")

# 개별 데이터 (전체 반응 텍스트 표시)
print("│ 【개별 데이터】" + " " * 63 + "│")
for player_id, result in multiplayer_results.items():
    player_info = next(p for p in players if p["player_id"] == player_id)
    heroine_name = player_info["heroine_data"]["name"]
    heroine_reaction_full = result["final_dungeon_json"]["events"]["heroine_reaction"]
    # 폭에 맞춰 줄바꿈 처리
    max_width = 70
    lines = []
    text = f"Player {player_id} ({heroine_name}): {heroine_reaction_full}"
    while len(text) > max_width:
        lines.append(text[:max_width])
        text = text[max_width:]
    lines.append(text)
    print("│   • " + lines[0] + " " * (78 - len("   • " + lines[0]) - 1) + "│")
    for line in lines[1:]:
        print("│     " + line + " " * (78 - len("     " + line) - 1) + "│")

print("└" + "─" * 78 + "┘")

print("\n\n✅ 멀티플레이어 테스트 체크리스트:")
print(f"   ✓ 총 {len(multiplayer_results)}명의 플레이어")
print(f"   ✓ 같은 던전 구조 공유 ✅")
print(f"   ✓ 같은 몬스터 배치 공유 ✅")
print(f"   ✓ 같은 메인 이벤트 텍스트 ✅")
print(f"   ✓ 같은 서브 이벤트 진행 ✅")
print(f"   ✓ 각 히로인별 개별 반응 텍스트 ✅")

# 검증
main_unique = len(
    set(
        [
            r["final_dungeon_json"]["events"]["main_event"]
            for r in multiplayer_results.values()
        ]
    )
)
sub_unique = len(
    set(
        [
            r["final_dungeon_json"]["events"]["sub_event"]
            for r in multiplayer_results.values()
        ]
    )
)
reaction_unique = len(
    set(
        [
            r["final_dungeon_json"]["events"]["heroine_reaction"]
            for r in multiplayer_results.values()
        ]
    )
)

print("\n\n🔍 데이터 검증:")
print(
    f"   • 메인 이벤트 고유성: {main_unique}/{len(multiplayer_results)} → {'✅ 공통' if main_unique == 1 else '❌ 개별'}"
)
print(
    f"   • 서브 이벤트 고유성: {sub_unique}/{len(multiplayer_results)} → {'✅ 공통' if sub_unique == 1 else '❌ 개별'}"
)
print(
    f"   • 히로인 반응 고유성: {reaction_unique}/{len(multiplayer_results)} → {'✅ 개별' if reaction_unique == len(multiplayer_results) else '❌ 공통'}"
)

print("\n" + "📊 " + "=" * 76 + " 📊")
print("                       검증 완료")
print("📊 " + "=" * 76 + " 📊")


📊 ============================================================================ 📊
                         최종 요약
📊 ============================================================================ 📊

┌──────────────────────────────────────────────────────────────────────────────┐
│                    멀티플레이어 이벤트 데이터 구조                            │
├──────────────────────────────────────────────────────────────────────────────┤
│ 【공통 데이터】                                                               │
│   • 던전 구조: 6개 방                                                     │
│   • 몬스터 배치: 3마리                                                      │
│   • 메인 이벤트: [미지의 기억]
Event Code: UNKNOWN_MEMORY
Type...                  │
│   • 서브 이벤트: 
=== 서브 이벤트 내러티브 ===
발을 들여놓는 순간, 방의 윤곽이 ...                  │
│                                                                              │
│ 【개별 데이터】                                                               │
│   • Player 101 (레티아): [레티아의 반응] 이 이벤트를 경험한 